# Social Data Science Exam 2019

Exam number: 202, 177, 43, 189

This is a project is about the housing market in Copenhagen focusing on the relationship between location of metro stations and the housing prices. The following code contains following elements:
- Scraping data and parsing location data
    - From Boliga 
    - For metro and S-train stations 
- Analysis 
    - Creating variables
    - Data validition
    - Statitical analysis
- Machine Learning

### Importing relevant modules and packages

In [ ]:
from folium import plugins
import folium
from selenium import webdriver
import time, os, glob, re
import pandas as pd
from Connector import Connector, ratelimit
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import random
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from scipy.stats import pearsonr
import matplotlib.dates as mdates
from geopy.distance import geodesic
from dateutil.parser import parse
from geopy.geocoders import Nominatim
import statsmodels
from tabulate import tabulate
from skmisc.loess import loess
from scipy.signal import savgol_filter
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso, LinearRegression, Ridge
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import validation_curve
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold

## Scraping data and parsing location
Scraping housing data from Boliga.dk and the station data from Wikipeadia

### Scraping data from Boliga

In [ ]:
# Initialization of selenium
path2gecko = '/Users/MacbookJos/git/geckodriver' # define path geckodriver
browser = webdriver.Firefox(executable_path=path2gecko)

# Direction to boliga.dk
browser.get('https://www.boliga.dk/salg/resultater?salesDateMin=1995&zipcodeFrom=1000&zipcodeTo=2499&searchTab=1&page=1&sort=date-a')
cookie_button = browser.find_element_by_xpath('//*[@id="coiAccept"]')
cookie_button.click()

# Defines function that process the data
def df2table(html):
    df = pd.read_html(html)[0]
    df = df.iloc[:, 0:8]
    
    # Translating and renaming columns
    for column in list(df.columns):
        df[column] = df[column].str.replace(column, '')
    df.columns = ["Address", "Sell_price", "Date_of_sale", "Type",
                  "sqm_price", "Rooms", "m2", "Building_year"]

    # Removing dublicates of the type of residence
    for i in range(len(df['Type'])):
        for j in ['EEjerlejlighed', 'VVilla', 'RRækkehus', 'FFritidshus']:
            df['Type'][i] = df['Type'][i].replace(j,'')

    # Changing the prices, so that it's numeric values
    for s in range(len(df['Sell_price'])):
        df['Sell_price'][s] = df['Sell_price'][s].replace('.','')
        df['sqm_price'][s] = df['sqm_price'][s].replace('.','')
    
    df['sqm_price'] = pd.to_numeric(df['sqm_price'])
    df['Sell_price'] = pd.to_numeric(df['Sell_price'])
    
    #Changing the date into date-time format
    def format_dates(date):
        q = date.split('-')
        return q[0].strip() + q[1].strip() + q[2]

    df['Date_of_sale'] = df['Date_of_sale'].apply(lambda x: format_dates(x))
    df['Date_of_sale'] = pd.to_datetime(df['Date_of_sale'], format='%d%m%Y')

    return df

i = 1
dir = str(os.getcwd())
while i <= 288: #288
    ## Initialization of log
    logfile = 'log_boliga_csv'
    project_name = 'SDS exam'
    header = ['id','project','connector_type','t', 'delta_t', 'url',\
              'redirect_url','response_size', 'response_code','success', 'error']

    if os.path.isfile(logfile):
        log = open(logfile,'a')
    else:
        log = open(logfile,'w')
        log.write(';'.join(header))

    ## load log
    with open(logfile,'r') as f: # open file
        l = f.read().split('\n') # read and split file by newlines.
        ## set id
        if len(l)<=1:
            id = 0
        else:
            id = int(l[-1][0])+1

    t = time.time()

    #Creating the log file
    """
    NOTE: several indicators are different than from the indicators
    that are established with the requests package. `dt`, does not
    necessarily reflect the complete load time. `size` might not be
    correct as selenium works in the background and could still be
    loading
    """
    html = browser.page_source
    df = df2table(html)

    ## Key arguments
    err = ''                           # define python error variable as empty assumming success.
    success = True                     # define success variable
    connector_type = "selenium"
    redirect_url = browser.current_url # log current url, after potential redirects
    dt = t - time.time()               # define delta-time waiting for the server and downloading content.
    size = df.memory_usage(deep=True, index=True).sum()    # define variable for size of html content of the response.
    response_code = ''                 # log status code.

    ## log...
    call_id = id                        # get current unique identifier for the call

    # Defines row to be written in the log
    row = [call_id, project_name, connector_type, t, dt,\
           redirect_url, size, response_code, success, err] # define row

    log.write('\n'+';'.join(map(str,row))) # write row to log file.
    log.flush()

    # storing parsed table as csv
    file_path = str(dir + '/boliga' +'/scrape' +'/%d.csv'%i)
    with open(file_path, mode='w', encoding='UTF-8',
              errors='strict', buffering=1) as f:
        f.write(df.to_csv())
    time.sleep(2)

    xpath_next = '/html/body/app-root/app-sold-properties-list/div[2]/app-pagination/div[1]/div[4]/a'
    next_button = browser.find_element_by_xpath(xpath_next)
    next_button.click()
    time.sleep(1)
    i += 1

########################    
# Apppending csv-files #
########################

#Path to file directory
boliga_directory = dir + '/boliga/scrape/'

#import all the files in the folder
boliga_files = glob.glob(os.path.join(boliga_directory, '*.csv'))

#loop throug all files and read as pandas
merged_df = [] #saves as list of dataframes
for boliga_file in boliga_files:
    df = pd.read_csv(boliga_file)
    merged_df.append(df)

#merge the dataframes with concat
boliga_data = pd.concat(merged_df, ignore_index=True)

#save as csv
file_path = str(dir + '/boliga/data/housing_data.csv')
with open(file_path, mode='w', encoding='UTF-8',
          errors='strict', buffering=1) as f:
    f.write(boliga_data.to_csv())

### Parsing location data onto Boliga data

In [ ]:
#Calling the geolocator from geopy
geolocator = Nominatim(user_agent="SDS Student")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1.1,
                       max_retries=3, error_wait_seconds=5,
                       return_value_on_exception=True)

## Defining a function which downloads information about longtitude and latitude
def address_parser (address_list):
    street =  []
    locations = []
    latitude = []
    longitude = []
    for addr in address_list:
        loc = geocode(addr)
        time.sleep(.2)
        if loc != None:
            lati = loc.latitude
            time.sleep(.2)
            long = loc.longitude
            time.sleep(.2)
            stre = loc.address
            time.sleep(.2)
        else:
            stre, lati, long = None, None, None
        latitude.append(lati)
        longitude.append(long)
        locations.append(stre)
    return address_list, locations, latitude, longitude

dir = str(os.getcwd())
datafile = dir + "/boliga/data/housing_data.csv"
boliga_data = pd.read_csv(datafile)

## cleaning DataFrame
boliga_data['Date_of_sale'] = pd.to_datetime(boliga_data['Date_of_sale'])
boliga_data.sort_values(by=['Date_of_sale'], inplace=True, ascending=True)
boliga_data = boliga_data.reset_index()
boliga_data = boliga_data.iloc[:, 3:]
df_address = pd.DataFrame([a[0] + ' ' + a[-1][6:]\
                           for a in boliga_data['Address'].str.split(',')])
df_address.drop_duplicates(inplace = True)
df_address[0] = df_address[0].str.lstrip()

 #check if folder exists
project = dir + '/boliga/location'
if not os.path.isdir(project):
     os.mkdir(project)

print(time.time())
 chunksize = 100
dt = []
for i, chunk in df_address.groupby(np.arange(len(df_address)) // chunksize):
    t = time.time()
    building_address, locations, latitude, longitude = address_parser(chunk[0])
    dt.append(t - time.time())
    dict = {'building_address':building_address,'location':locations, 'latitude':latitude, 'longitude': longitude}
    df = pd.DataFrame(dict)
    file_path = dir + "/boliga/location/location_data_" + str(i) + ".csv"
    with open(file_path, mode='w', encoding='UTF-8',
                   errors='strict', buffering=1) as f:
        f.write(df.to_csv())
print(time.time())

file_path = dir + "/boliga/location/location_data_log"
with open(file_path, mode='w', encoding='UTF-8',
          errors='strict', buffering=1) as f:
    f.write(dt)
    
#Transforming into a pandas dataframe
#Path to file directory
location_dir = dir + '/boliga/location/'

#import all the files in the folder
location_files = glob.glob(os.path.join(location_dir, '*.csv'))

#loop throug all files and read as pandas
merged_df = [] #saves as list of dataframes
for file in location_files:
    df = pd.read_csv(file)
    merged_df.append(df)

 #merge the dataframes with concat
 location_data = pd.concat(merged_df, ignore_index=True)

#save as csv
file_path = str(dir + '/boliga/data/location_data.csv')
with open(file_path, mode='w', encoding='UTF-8',
          errors='strict', buffering=1) as f:
    f.write(location_data.to_csv())

### Scraping station data and parsing location data onto them 

In [ ]:
##################
# Metro stations #
##################

#Logfile of metro stations
logfile = 'log_station_scrape.csv'## name your log file.
connector = Connector(logfile)

# Scrape initial table of stations
url = "https://en.wikipedia.org/wiki/List_of_Copenhagen_Metro_stations"
resp, callid = connector.get(url, 'mstation_scrape')
html = resp.text

# parse table
table = pd.read_html(resp.text)[1]
table = table.drop(['Transfer', 'Line'], axis = 1)
table['Station'] = table['Station']

# Importing meta-data on stations
def links (html):
    s = BeautifulSoup(html, 'html.parser')
    table_html = s.find_all('table')[1]
    urlFmt= re.compile('href="(\S*)"')
    link_locations = urlFmt.findall(str(table_html))
    links = ['https://en.wikipedia.org' + i for i in link_locations]
    links = [link for link in links if '/wiki/' in link]
    for i in ['S-train', 'M3_', 'M4_', 'M2_', 'M1_', 'S-train', 'Template',
              'K%C3%B8ge_Nord','File:', 'List', 'commons.', '_Metro', '_Line',
              'Airport', 'Lokaltog', 'Letbane']:
        links = [link for link in links if not i in link]
    return sorted(list(set(links)))

# Function whih gets lontitutes and latitudes of stations
def mstation_location(urls):
    stations = []
    longitudes = []
    latitudes = []
    for station in urls:
        html = requests.get(station).text
        s = BeautifulSoup(html, 'lxml')
        loc = s.select('span.geo-dms')[0]
        latFmt = re.compile('.*latitude">(.*)</span> <')
        lonFmt = re.compile('.*longitude">(.*)</span><')
        longitudes.append(lonFmt.findall(str(loc)))
        latitudes.append(latFmt.findall(str(loc)))
        stations.append(str(s.title.string).replace(' Station - Wikipedia', ''))
    return stations, longitudes, latitudes

mstation, longitude, latitude = mstation_location(links(html))

# Parsing location data in degrees with decimals
def dms2dd(degrees, minutes, seconds, direction):
    dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60);
    if direction == 'E' or direction == 'N':
        dd *= -1
    return dd;

def dd2dms(deg):
    d = int(deg)
    md = abs(deg - d) * 60
    m = int(md)
    sd = (md - m) * 60
    return [d, m, sd]

def parse_dms(dms):
    parts = re.split('[°′″]+', str(dms))
    lat = dms2dd(parts[0], parts[1], parts[2], parts[3])
    return (lat)

# Defining the coordinates for the location of the stations
latitude = [parse_dms(lat[0]) for lat in latitude]
longitude = [parse_dms(long[0]) for long in longitude]

# Convert and merge into a pandas dataframe
df_location = pd.DataFrame([mstation, longitude, latitude]).T
df_location.columns = ['Station', 'Longitude', 'Latitude']
df_mstation = pd.merge(table, df_location, on='Station')

# store the metro stations
file_path = dir + "/boliga/data/mstation_data.csv"
with open(file_path, mode='w', encoding='UTF-8',
              errors='strict', buffering=1) as f:
    f.write(df_mstation.to_csv())

####################
# S-train stations #
####################

# Scrape initial table of stations
url = "https://en.wikipedia.org/wiki/List_of_Copenhagen_S-train_stations"
resp, callid = connector.get(url, 'sstation_scrape')
html = resp.text

# Parse table
table = pd.read_html(resp.text)[1]
table = table.drop(['Transfer', 'Line'], axis = 1)
table['Station'] = table['Station'].str.translate({ord('#'): '', ord('†'): ''})

# Function whih gets lontitutes and latitudes of stations
def sstation_location(urls):
    stations = []
    longitudes = []
    latitudes = []
    for station in urls:
        html = requests.get(station).text
        s = BeautifulSoup(html, 'lxml')
        loc = s.select('span.geo-dms')[0]
        latFmt = re.compile('.*latitude">(.*)</span> <')
        lonFmt = re.compile('.*longitude">(.*)</span><')
        station = str(s.title.string).replace(' Station - Wikipedia', '')
        station = station.replace(' station - Wikipedia', '')
        #appending to containers
        longitudes.append(lonFmt.findall(str(loc)))
        latitudes.append(latFmt.findall(str(loc)))
        stations.append(station)
    return stations, longitudes, latitudes

sstation, longitude, latitude = sstation_location(links(html))

# Defining the coordinates for the location of the stations
latitude = [parse_dms(lat[0]) for lat in latitude]
longitude = [parse_dms(long[0]) for long in longitude]

# Convert and merge into a pandas dataframe
df_location = pd.DataFrame([sstation, longitude, latitude]).T
df_location.columns = ['Station', 'Longitude', 'Latitude']
df_sstation = pd.merge(table, df_location, on='Station')

# Shows the s-train stations
file_path = dir + "/boliga/data/sstation_data.csv"
with open(file_path, mode='w', encoding='UTF-8',
              errors='strict', buffering=1) as f:
    f.write(df_sstation.to_csv())

### Random sample inspection of our data

In [ ]:
# Random sample of the data scraped from boliga.dk
print(boliga_data.sample(15))

# Random sample of data from the metro stations
print(df_mstation.sample(5))

# Random sample of data from the s-train stations
print(df_sstation.sample(5))

# Checks if our scraped data contains duplicates
# Boliga constains 14.400 sold houses in the chosen period of time (01/01/1995 - 31/12/2007)
print(len(boliga_data), len(boliga_data.drop_duplicates()))

# Shows the amount of different real estate types (appartments, houses, terraced house)
print(boliga_data['Type'].value_counts())

### Merging full data

In [ ]:
# load boliga_data
dir = str(os.getcwd())
datafile = dir + "/boliga/data/housing_data.csv"
boliga_data = pd.read_csv(datafile)
boliga_data = boliga_data.iloc[:, 2:]
boliga_data['building_address'] = pd.DataFrame([a[0] + ' ' + a[-1][6:]\
                for a in boliga_data['Address'].str.split(',')])
boliga_data['building_address'] = boliga_data['building_address'].str.lstrip()

# load appended location data
location_file = dir +"/boliga/data/location_data.csv"
location_data = pd.read_csv(location_file)
location_data = location_data.iloc[:,2:]

#merge data
full_data = pd.merge(boliga_data, location_data, on= 'building_address')
full_data = full_data.reset_index()
full_data = full_data.drop(['index', 'building_address'], axis=1)
print(full_data.head())

file_path = str(dir + '/boliga/data/full_data.csv')
with open(file_path, mode='w', encoding='UTF-8',
          errors='strict', buffering=1) as f:
    f.write(full_data.to_csv())

### Loading analysis data from full data

In [ ]:
#Loading in csv-files
dir = str(os.getcwd())
path_full = dir +'/boliga/data/full_data.csv'
path_msta = dir +'/boliga/data/mstation_data.csv'
path_ssta = dir +'/boliga/data/sstation_data.csv'
full_data = pd.read_csv(path_full, index_col=0)
df_mstation = pd.read_csv(path_msta, index_col=0)
#Correcting the negative values to a positiv
df_mstation[['Longitude','Latitude']] = df_mstation[['Longitude','Latitude']].abs()
df_sstation = pd.read_csv(path_ssta, index_col=0)
#Correcting the negative values to a positiv
df_sstation[['Longitude', 'Latitude']] = df_sstation[['Longitude','Latitude']].abs()

## Data analysis
In this section, the code for the data quality, descriptive statistics, the analysis is to be found.

### Sample selection

In [ ]:
#sample identification
#sample should be from befor 2008
full_data = full_data.loc[full_data['Date_of_sale'] <= '2007-12-31']

# deleting outliers
full_data = full_data.loc[full_data['sqm_price']<80000]
full_data = full_data.loc[full_data['Sell_price']<10000000]
print('missingness on address: ',round((full_data.iloc[:, -1].isna().mean())*100,2), "%")
print('missingness on address: ',full_data.iloc[:, -1].isna().sum())
full_data = full_data[full_data['longitude']>12]

full_data = full_data.sort_values('Date_of_sale')
full_data = full_data.reset_index(drop=True)

### Data Quality

In [ ]:
# Random sample of the data scraped from boliga.dk
print("random sample of final data, \n",full_data.sample(100))

# Random sample of data from the metro stations
print("m-train station, \n", df_mstation)

# Random sample of data from the s-train stations
print("s-train station, \n", df_sstation)

# Checking for duplicates
print("checking for duplicates: ",len(full_data), len(full_data.drop_duplicates()))

# Shows the amount of different real estate types (appartments, houses, terraced house)
print("count py real estate types: ", full_data['Type'].value_counts())

###  Visualization of log file

In [ ]:
# Loading the logfile as a pandas dataframe
log_df = pd.read_csv('log_boliga.csv', sep=';')
log_df = log_df.iloc[:,:-2]
log_df = log_df.drop(['response_size'], axis=1)
log_df['dt'] = pd.to_datetime(log_df['t'], unit= 's')
log_df.sort_values('id')
log_df.columns = ['id', 'project', 'connector_type', 't', 'delta_t', 'url',
                  'response_size', 'response_code', 'datetime' ]
print("head of log_file: \n", log_df.head())

# Visualization of the time it took to make the call for data
plt.figure(figsize = (13, 4))
plt.plot(log_df['datetime'], log_df.delta_t, color='green')
plt.ylabel('Delta t', color='black')
plt.xlabel('Scraping process', color='black')
plt.title('a: The time it took to make the call for data', weight='bold')
plt.savefig('fig4.png')

# Visualization of the response size through the scraping process
plt.figure(figsize = (6.5, 4))
plt.plot(log_df['datetime'], log_df['response_size'], color='green')
plt.ylabel('bytes in the csv files', color='black')
plt.xlabel('Scraping process', color='black')
plt.title('b: The size of csv files through the scraping process', weight='bold')
plt.savefig('fig5.png')

# Plot the delta_t against the response_size - to see correlation.
fig, ax = plt.subplots(figsize = (5, 4))
ax.scatter(log_df.delta_t.astype('float'), log_df.response_size, color = 'green')
ax.fmt_xdata = mdates.DateFormatter('%s-%f')
plt.ylabel('')
plt.xlabel('Delta t', color='black')
plt.title('c: assocation between Delta t and file size', weight='bold')
plt.savefig('fig6.png')

q_cols = log_df.loc[:, ['delta_t', 'response_size']]
print(q_cols.corr(method='pearson'))

### Descriptive statistics

In [ ]:
totalsum = np.sum(full_data['Type'].value_counts())
ListOfTypes = ['Ejerlejlighed', 'Villa', 'Rækkehus','Fritidshus']
for i in ListOfTypes:
    cd = full_data['Type'].value_counts()
    for i in cd:
        percent_type = round(cd/totalsum ,3)

percent_df = pd.DataFrame(percent_type)

# Rename columns
percent_df = percent_df.rename(columns={'Type':'Share of total'})
print("descriptive statistics: \n", percent_df)

# Histogram of the distrubution of square meters
fig1 = plt.hist(full_data['m2'], color = 'green', edgecolor = 'black', bins=40, linewidth=1.2)
fig1 = plt.title('a: Size of sold real-estate properties', weight='bold')
fig1 = plt.xlabel('Square meters', color='black')
fig1 = plt.ylabel('Observations', color='black')
fig1 = plt.xlim(0,420)
fig1 = plt.ylim(0,3500)
plt.savefig('fig1.png')

# Rooms in sold properties
fig2 = plt.hist(full_data['Rooms'], color = 'green', edgecolor='black', bins=20, linewidth=1.2)
fig2 = plt.title('b: Rooms in sold real-estate properties', weight='bold')
fig2 = plt.xlabel('Rooms', color='black')
fig2 = plt.ylabel('Observations', color='black')
fig2 = plt.xlim(0,12)
fig2 = plt.ylim(0,5000)
plt.savefig('fig2.png')

# Building year of sold properties
fig3 = plt.hist(full_data['Building_year'], bins=90, color= 'green', edgecolor='black', linewidth = 1.2)
fig3 = plt.title('Building year of real-estate properties', weight='bold')
fig3 = plt.xlabel('Year', color='black')
fig3 = plt.ylabel('Observations', color='black')
fig3 = plt.xlim(1600, 2030)
fig3 = plt.ylim(0, 3500)
plt.savefig('fig3.png')

### Montly changes

In [ ]:
#Preparing DataFrame with monthly change
dta = full_data['Date_of_sale']
dta1 = pd.to_datetime(dta)
merged1 = pd.concat([dta1, full_data['sqm_price']], axis=1)
merged1 = merged1.set_index('Date_of_sale')

monthly_price = round(merged1['sqm_price'].resample('M').mean(),2)
monthly_price = monthly_price.to_frame()

monthly_price['Monthly change'] = round(monthly_price.pct_change(periods=1, axis=0),2)
def percent_change(df):
    monthly_price['Total change'] = round((100 * (monthly_price.sqm_price/monthly_price.iloc[0].sqm_price -1)),2)
    return monthly_price

monthly_price.groupby('Date_of_sale').apply(percent_change)
monthly_price.index = pd.to_datetime(monthly_price.index)

#Figure 9
fig, ax = plt.subplots()
ax.plot(monthly_price['Monthly change'], color='green')
plt.title('a: Monthly change in sqare meter price', weight = 'bold')
plt.xlabel('Year', color='black')
plt.ylabel('Change, %', color='black')
fig.autofmt_xdate()
ax.fmt_xdata = mdates.DateFormatter('%Y-%m-%d')
plt.savefig('fig9.png')

# Figure 10
fig, ax = plt.subplots()
ax.plot(monthly_price['Total change'], color ='green')
plt.title('b: Total change in square meter price since 1995', weight='bold')
plt.xlabel('Year', color='black')
plt.ylabel('Change, %', color='black')
fig.autofmt_xdate()
ax.fmt_xdata = mdates.DateFormatter('%Y-%m-%d')
plt.savefig('fig10.png')

### Table summerizing changes and averages

In [ ]:
yearly = round(merged1['sqm_price'].resample('Y').mean(),2)
yearly = yearly.to_frame()

# Average percentual change 1995-2002:
av1 = round(((16979.78/6349.07)**(1/7)-1)*100,1)
print(av1)

#Average percentual change 2002 - 2007
av2 = round(((28184.09/17691.70)**(1/4)-1)*100,1)
print(av2)

#Total
av3 = round((28184.09/6349.07-1)*100,1)

# Average percentual change 2003-2006
av4 = round(((30660.32/17691.70)**(1/3)-1)*100,1)
print(av3)
print(av4)

#Average square meters
av5 = round(sum(full_data['m2'])/12542,1)
print(av5)

#Average rooms
av6 = round(sum(full_data['Rooms'])/12542,1)

table1 = [["1995-2002",15.1,"NaN"],["2003-2007",12.4,"NaN"],["2002-2006",20.1,"NaN"],
          ["Total",343.9,"NaN"], ["Average sqm", "NaN", 3.4],["Average rooms", "NaN", 15.1]]
print(tabulate(table1, headers=["pct. change","Average"], tablefmt="latex"))

ab2 = full_data.drop(['Unnamed: 0', 'Address','Date_of_sale', 'location','m_station_const','s_station','m_station','s_station_const','latitude','longitude','Building_year'], axis=1)
a1 = round(ab2.describe().T,1)
a1=a1.reset_index()

#importing plotly
import plotly.graph_objects as go
import os

#making a folder called images:
if not os.path.exists("images"):
    os.mkdir("images")
    
#making a table figure
fig14 = go.Figure(data=[go.Table(columnwidth = [650,400],
                                 header=dict(values=['Variable','Count','Mean','Std','Min','Max'], 
                                            line_color='black',
                                            fill_color='lightgrey',
                                            font=dict(color='black', size=14),
                                            align=['left','center']),
                                 cells=dict(values=[a1['index'],a1['count'],a1['mean'],a1['std'],a1['min'],a1['max']], 
                                            line_color='black',
                                            fill=dict(color=['lightgrey', 'white']),
                                            font = dict(color = 'Black', size = 12),
                                            align=['left', 'center']))
                       ])

fig14.write_image("images/fig14.png")

### Deleting outliers

In [ ]:
# Sample identification
# Sample should be from befor 2008
full_data = full_data.loc[full_data['Date_of_sale'] <= '2007-12-31']

# Deleting outliers
full_data = full_data.loc[full_data['sqm_price']<80000]
full_data = full_data.loc[full_data['Sell_price']<10000000]
print('missingness on address: ',round((full_data.iloc[:, -1].isna().mean())*100,2), "%")
full_data = full_data[full_data['longitude']>12]

full_data = full_data.sort_values('Date_of_sale')
full_data = full_data.reset_index(drop=True)
print(full_data.shape)

### Creating heatmap 

In [ ]:
#folium map

def map_points(df, lat_col='latitude', lon_col='longitude', zoom_start=11, \
                plot_points=False, pt_radius=2, \
                draw_heatmap=False, heat_map_weights_col=None, \
                heat_map_weights_normalize=True, heat_map_radius=15):
    """Creates a map given a dataframe of points. Can also produce a heatmap overlay
    Arg:
        df: dataframe containing points to maps
        lat_col: Column containing latitude (string)
        lon_col: Column containing longitude (string)
        zoom_start: Integer representing the initial zoom of the map
        plot_points: Add points to map (boolean)
        pt_radius: Size of each point
        draw_heatmap: Add heatmap to map (boolean)
        heat_map_weights_col: Column containing heatmap weights
        heat_map_weights_normalize: Normalize heatmap weights (boolean)
        heat_map_radius: Size of heatmap point
    Returns:
        folium map object
    """

    ## center map in the middle of points center in
    middle_lat = df[lat_col].median()
    middle_lon = df[lon_col].median()

    curr_map = folium.Map(location=[middle_lat, middle_lon],
                          zoom_start=zoom_start,
                          tiles='Stamen Terrain')

    # add points to map
    if plot_points:
        for _, row in df.iterrows():
            folium.CircleMarker([row[lat_col], row[lon_col]],
                                radius=pt_radius,
                                popup=row['location'],
                                fill_color="#3db7e4", # divvy color
                               ).add_to(curr_map)

    # add heatmap
    if draw_heatmap:
        # convert to (n, 2) or (n, 3) matrix format
        if heat_map_weights_col is None:
            cols_to_pull = [lat_col, lon_col]
        else:
            # if we have to normalize
            if heat_map_weights_normalize:
                df[heat_map_weights_col] = \
                    df[heat_map_weights_col] / df[heat_map_weights_col].sum()

            cols_to_pull = [lat_col, lon_col, heat_map_weights_col]

        houses = df[cols_to_pull].values
        curr_map.add_child(plugins.HeatMap(houses, radius=heat_map_radius,gradient={.4: 'green', .85:'yellow', 1: 'red'}))


    return curr_map

# define map
# Creating a map of Copenhagen (specified to the area that we need)
m = folium.Map([55.676098, 12.568337], zoom_start=12)

# Adding s-train stations to the map
for index, row in df_sstation.iterrows():
    folium.Circle(
        radius=50,
        location=(row['Latitude'],row['Longitude']),
        popup=row['Station'],
        fill=True,
        color = 'Blue').add_to(m)

# Adding metro-stations to the map
for index, row in df_mstation.iterrows():
    folium.Circle(
        radius=50,
        location=(row['Latitude'],row['Longitude']),
        popup=row['Station'],
        fill=True,
        color = 'Red').add_to(m)

m

# Adding the heatmap of sold properties to the map with s-train and metro-stations
Sold_houses = full_data[['latitude', 'longitude']].values
m.add_child(plugins.HeatMap(Sold_houses, radius=10))

Train_legend =   '''
                <div style="position: fixed; background:white;
                            top: 150px; right: 350px; width: 170px; height: 75;
                            border:3px solid grey; z-index:9999; font-size:16.5px; font-color=black; weight='bold';
                            ">&nbsp; Station type: <br>
                              &nbsp; Metro-station &nbsp; <i class="fa fa-circle-thin fa-1x" style="color:red"></i><br>
                              &nbsp; S-train station &nbsp; <i class="fa fa-circle-thin fa-1x" style="color:blue"></i>
                </div>
                '''

m.get_root().html.add_child(folium.Element(Train_legend))
m.save('copenhagen_map.html')

# using selenium to safe HTML as png
path2gecko = '/Users/MacbookJos/git/geckodriver' # define path  geckodriver
browser = webdriver.Firefox(executable_path=path2gecko)
html= 'file:///Users/MacbookJos/git/sds_exam/copenhagen_map.html'

browser.get(html)
time.sleep(5)
browser.save_screenshot('fig8.png')
browser.quit()

### Creating list of neigbourhood

In [ ]:
address = pd.DataFrame([a[0] + ' ' + a[-1][6:]\
           for a in full_data['Address'].str.split(',')])
header=['ad']
address.columns=header

municipalities = ['Frederiksberg C', 'Frederiksberg', 'København K', 'København V', 'København S', 'København N', 'København Ø']

address_list = []

for i in address['ad']:
    for x in municipalities:
        if x in i:
            address_list.append(x)
            break

# Making a list of municipalities
Municipality = pd.DataFrame(address_list)
Municipality.columns=['Municipality']

# Adding the list to the housing data
df['Municipality'] = Municipality
df = df[['Municipality', 'Type']]
df = pd.get_dummies(df)
df.describe().round(3)


### Calculating distances

In [ ]:
# #parse the opening year from int to string to datetime
df_sstation['Opened'] = df_sstation['Opened'].apply(lambda x: str(x))
df_mstation['Opened'] = df_mstation['Opened'].apply(lambda x: str(x))
df_sstation['Opened'] = df_sstation['Opened'].apply(lambda x: parse(x))
df_mstation['Opened'] = df_mstation['Opened'].apply(lambda x: parse(x))
#parse the date of sale from to datetime
full_data['Date_of_sale'] = full_data['Date_of_sale'].apply(lambda x: str(x))
full_data['Date_of_sale'] = full_data['Date_of_sale'].apply(lambda x: parse(x))

#find which stations were open at the year of the sale
def was_opened(property_, stations):
    open_stations = {}
    for i in range(len(stations)):
        if (stations.iat[i,2] < property_[2]) == True:
            open_stations[stations.iat[i,0]] = (abs(stations.iat[i,-1]), abs(stations.iat[i,-2]))
    return open_stations

#calculate distance to closeste open station
def get_distance_opened(property_, df_stations):
    stations = was_opened(property_, df_stations)
    if  stations == {}:
        return None, None
    property_loc = (property_[-2], property_[-1])
    if property_loc[0] is not None or property_loc[1] is not None:
        min_dist = 999999999999999999999
        min_dist_station = ''
        for station, station_loc in stations.items():
            dist = geodesic(station_loc, property_loc).km
            if dist < min_dist:
                min_dist = dist
                min_dist_station = station
        return round(min_dist,5), min_dist_station
    else:
        return None, None

# find which stations which were not open during the time of the sale 
def was_not_opened(property_, stations):
    open_stations = {}
    for i in range(len(stations)):
         if (stations.iat[i,2] > property_[2]) == True:
            open_stations[stations.iat[i,0]] = (abs(stations.iat[i,-1]),abs(stations.iat[i,-2]))
    return open_stations

#Calculates the distance to the closest station under construction
def get_distance_not_opened(property_, df_stations):
    stations = was_not_opened(property_, df_stations)
    if  stations == {}:
        return None, None
    property_loc = (property_[-2], property_[-1])
    if property_loc[0] is not None or property_loc[1] is not None:
        min_dist = 999999999999999999999
        min_dist_station = ''
        for station, station_loc in stations.items():
            dist = geodesic(station_loc, property_loc).km
            if dist < min_dist:
                min_dist = dist
                min_dist_station = station
        return round(min_dist,3), min_dist_station
     else:
        return None, None

#calculates the distance to the city center     
geolocator = Nominatim(user_agent="Social Data Science Student", timeout =50)
copenhagen = geolocator.geocode("Copenhagen")[-1]
def dist_city_center(property_):
    property_loc = (property_[-2], property_[-1])
    if property_loc[0] is not None or property_loc[1] is not None:
        return round(geodesic(property_loc, copenhagen).km, 3)
    else:
        return None

# construct distance objects
distance_m = full_data.apply(lambda x: get_distance_opened(x, df_mstation), axis=1)
distance_s = full_data.apply(lambda x: get_distance_opened(x, df_sstation), axis=1)
distance_m_const = full_data.apply(lambda x: get_distance_not_opened(x, df_mstation), axis=1)
distance_s_const = full_data.apply(lambda x: get_distance_not_opened(x, df_sstation), axis=1)
distance_c = full_data.apply(dist_city_center, axis=1)

#adding the distances to the dataframe
full_data['m_distance'], full_data['m_station'] = [m[0]for m in distance_m], [m[1]for m in distance_m]
full_data['s_distance'], full_data['s_station'] = [s[0]for s in distance_s], [s[1]for s in distance_s]
full_data['m_distance_const'], full_data['m_station_const'] = [m[0]for m in distance_m_const], [m[1]for m in distance_m_const]
full_data['s_distance_const'], full_data['s_station_const'] = [s[0]for s in distance_s_const], [s[1]for s in distance_s_const]
full_data['c_distance'] = distance_c

 # safe file
file_path = dir + "/boliga/data/analysis_data.csv"
with open(file_path, mode='w', encoding='UTF-8',
              errors='strict', buffering=1) as f:
    f.write(full_data.to_csv())
path_analysis = dir +'/boliga/data/analysis_data.csv'
full_data = pd.read_csv(path_analysis, index_col=0)

### Construction a percentage difference from  a rolling mean

In [ ]:
full_data['z_sqm_price'] = (full_data.sqm_price -\
                            full_data.sqm_price.rolling(window=30).mean())\
                            / full_data.sqm_price.rolling(window=30).mean()*100
 # safe file
file_path = dir + "/boliga/data/analysis_data.csv"
with open(file_path, mode='w', encoding='UTF-8',
              errors='strict', buffering=1) as f:
    f.write(full_data.to_csv())
path_analysis = dir +'/boliga/data/analysis_data.csv'
full_data = pd.read_csv(path_analysis, index_col=0)

# ensuring date time as column type
full_data['Date_of_sale'] = full_data['Date_of_sale'].apply(lambda x: str(x))
full_data['Date_of_sale'] = full_data['Date_of_sale'].apply(lambda x: parse(x))

### Making plots

In [ ]:
# make distance plot
datafile = dir + "/boliga/data/analysis_data.csv"
df = pd.read_csv(datafile, index_col=0)
df = df[['z_sqm_price', 'm_distance']]
df = df.dropna()
df.sort_values('z_sqm_price')

x = df['m_distance'].to_numpy()
y = df['z_sqm_price'].to_numpy()
sorted_indices = np.argsort(x)
sorted_x = x[sorted_indices]

l = loess(x, y, frac=0.1)
l.fit()
pred = l.predict(sorted_x, stderror=True)
conf = pred.confidence()

lowess = pred.values
ll = conf.lower
ul = conf.upper

# plt.plot(x, y, '+')
fig, ax1 = plt.subplots( figsize=(13, 4))
ax1.plot(sorted_x, lowess, color='green')
ax1.fill_between(sorted_x,ll,ul,alpha=.15, color='green')
ax1.set_ylabel('%-difference in square meter price from rolling mean')
ax.set_xlabel("distance in kilometers")
ax1.set_xlim(0,5)
plt.savefig('fig7.png')

### Making opening plot points 

In [ ]:
# list of opning years for all metro stations
m2002 = ['Sundby', 'Ørestad', 'Nørreport', 'Lergravsparken', 'Kongens Nytorv', 'Islands Brygge', 'DR Byen', 'Bella Center', 'Amagerbro']
m2003 = ['Fasanvej', 'Forum', 'Frederiksberg', 'Lindevang']
m2004 = ['Flintholm']
m2007 = ['Amager Strand', 'Øresund','Femøren', 'Kastrup']

# making new dataframes for each opening year for those sold after metro opening
open2002 = full_data.loc[full_data['m_station'].isin(m2002)]
open2003 = full_data.loc[full_data['m_station'].isin(m2003)]
open2004 = full_data.loc[full_data['m_station'].isin(m2004)]
open2007 = full_data.loc[full_data['m_station'].isin(m2007)]

# making new dataframes for each opening year for those sold before metro opening
open2002_c = full_data.loc[full_data['m_station_const'].isin(m2002)]
open2003_c = full_data.loc[full_data['m_station_const'].isin(m2003)]
open2004_c = full_data.loc[full_data['m_station_const'].isin(m2004)]
open2007_c = full_data.loc[full_data['m_station_const'].isin(m2007)]

open2002_c['m_distance'] = open2002_c['m_distance_const']
open2003_c['m_distance'] = open2003_c['m_distance_const']
open2004_c['m_distance'] = open2004_c['m_distance_const']
open2007_c['m_distance'] = open2007_c['m_distance_const']

# join the two data set 
open2002 = open2002.append(open2002_c, ignore_index=True)
open2003 = open2003.append(open2003_c, ignore_index=True)
open2004 = open2004.append(open2004_c, ignore_index=True)
open2007 = open2007.append(open2007_c, ignore_index=True)

#function to group distances in three categories
def group_dist(dist):
    if dist < 0.5:
        x = 'Less than 0.5 km'
    elif dist >= 0.5 and dist <= 1:
        x = '0.5 -1 km'
    else:
        x = 'Above 1 km'
    return x

#grouping distances 
dist_group2002 = open2002['m_distance'].apply(lambda x: group_dist(x))
dist_group2003 = open2003['m_distance'].apply(lambda x: group_dist(x))
dist_group2004 = open2004['m_distance'].apply(lambda x: group_dist(x))
dist_group2007 = open2007['m_distance'].apply(lambda x: group_dist(x))
open2002.insert(1, 'Grouped_distance', dist_group2002)
open2003.insert(1, 'Grouped_distance', dist_group2003)
open2004.insert(1, 'Grouped_distance', dist_group2004)
open2007.insert(1, 'Grouped_distance', dist_group2007)

#getting data per year
open2002['Date_of_sale'] = open2002['Date_of_sale'].apply(lambda x: x.year)
open2003['Date_of_sale'] = open2003['Date_of_sale'].apply(lambda x: x.year)
open2004['Date_of_sale'] = open2004['Date_of_sale'].apply(lambda x: x.year)
open2007['Date_of_sale'] = open2007['Date_of_sale'].apply(lambda x: x.year)


#creating plot for all four years 
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, figsize=(13, 20))
plt.tight_layout(pad=3, w_pad=2.5, h_pad=2)
sns.lineplot(data=open2002, x='Date_of_sale', y='z_sqm_price', hue='Grouped_distance', palette='Greens', ax=ax1)
ax1.axvline(2002, color='k', linestyle=':')
ax1.set_xlabel("Year which property was sold")
ax1.set_ylabel("%-difference in square meter price from rolling mean")
ax1.margins(0)
sns.lineplot(data=open2003, x='Date_of_sale', y='z_sqm_price', hue='Grouped_distance', palette='Greens', ax=ax2)
ax3.axvline(2004, color='k', linestyle=':')
ax3.set_xlabel("Year which property was sold")
ax3.set_ylabel("%-difference in square meter price from rolling mean")
ax3.margins(0)
sns.lineplot(data=open2004, x='Date_of_sale', y='z_sqm_price', hue='Grouped_distance', palette='Greens', ax=ax3)
ax2.axvline(2003, color='k', linestyle=':')
ax2.set_xlabel("Year which property was sold")
ax2.set_ylabel("%-difference in square meter price from rolling mean")
ax2.margins(0)
sns.lineplot(data=open2007, x='Date_of_sale', y='z_sqm_price', hue='Grouped_distance', palette='Greens', ax=ax4)
ax4.axvline(2007, color='k', linestyle=':')
ax4.set_xlim(1998,2008)
ax4.set_xlabel("Year which property was sold")
ax4.set_ylabel("%-difference in square meter price from rolling mean")
ax4.legend(loc=1)
ax4.margins(0)
plt.savefig('fig11.png')

#creating plot for 2002 
fig, ax = plt.subplots(figsize = (13, 4))
sns.lineplot(data=open2002, x='Date_of_sale', y='z_sqm_price', hue='Grouped_distance', palette='Greens', ax=ax)
# plt.title('%-difference in square meter price time from opening date', weight="bold")
ax.axvline(2002, color='k', linestyle=':')
ax.legend(loc=4)
ax.set_xlabel("Year which property was sold")
ax.set_ylabel("%-difference in square meter price from rolling mean")
plt.ylim(-30,20)
plt.margins(0)
plt.savefig('fig11_2002.png')


### Creating boxplots

In [ ]:
#creating a new column for minimum distance for any station
full_data['any_distance'] = full_data[['m_distance', 's_distance']].apply((lambda x: min(x)), axis=1)

#create new columns at our dataframe which check the distance to the nearest station
full_data['less_than_500m_mstation'] = full_data['m_distance'] <= 0.5
full_data['less_than_500m_sstation'] = full_data['s_distance'] <= 0.5
full_data['less_than_500m_any_station'] = full_data['any_distance'] <= 0.5
full_data['more_than_500m_less_than_1km_mstation'] = ((full_data['m_distance'] > 0.5) & (full_data['m_distance'] <= 1.0))
full_data['more_than_500m_less_than_1km_sstation'] = ((full_data['s_distance'] > 0.5) & (full_data['s_distance'] <= 1.0))
full_data['more_than_500m_less_than_1km_any_station'] = ((full_data['any_distance'] > 0.5) & (full_data['any_distance'] <= 1.0)
full_data['more_than_1km_mstation'] = full_data['m_distance'] > 1
full_data['more_than_1km_sstation'] = full_data['s_distance'] > 1
full_data['more_than_1km_any_station'] = full_data['any_distance'] > 1

#keep only data after 2002; the year that the first metro stations were built
data_after_2002 = full_data[full_data['Date_of_sale'] >= '2002-01-01']
                                                         
#creates 2 new columns in our dataframe which divide the houses in three categories (1,2,3) 
mdist = []
sdist = []
adist = []                                               
for i in range(len(data_after_2002['less_than_500m_mstation'])):
    #checking distance to mstations and creates a list of 3 categories
    if data_after_2002['less_than_500m_mstation'].iloc[i] == True:
        mdist.append(1)   
    elif data_after_2002['more_than_500m_less_than_1km_mstation'].iloc[i] == True:
        mdist.append(2)
    else:
        mdist.append(3)
        
    #checking distance to sstations and creates a list of 3 categories
    if data_after_2002['less_than_500m_sstation'].iloc[i] == True:
        sdist.append(1)   
    elif data_after_2002['more_than_500m_less_than_1km_sstation'].iloc[i] == True:
        sdist.append(2)
    else:
        sdist.append(3)
                                                         
    #checking distance to any_stations and creates a list of 3 categories
    if data_after_2002['less_than_500m_any_station'].iloc[i] == True:
        sdist.append(1)   
    elif data_after_2002['more_than_500m_less_than_1km_any_station'].iloc[i] == True:
        sdist.append(2)
    else:
        sdist.append(3)



#depending on their distance from the nearest station
data_after_2002['R_to_mstation'] = mdist
data_after_2002['R_to_sstation'] = sdist
data_after_2002['R_to_any_station'] = adist 

                                                        
# creating boxplots for metro station
f, ax = plt.subplots(figsize=(10,6))
sns.boxplot(x = 'R_to_mstation', y = 'sqm_price', data = data_after_2002, palette="Greens");
plt.xlabel('Residence distance to M-station', fontsize=10);
plt.ylabel('sqm price (DKK)', fontsize=10);

plt.xticks([0, 1, 2], ['r < 0.5 km', 'r > 0.5km and r < 1 km', 'r > 1 km']);
f.tight_layout()
f.savefig('sqm_distance_mstations.png');

# creating boxplots for S-train stations                                                                                     
f, ax = plt.subplots(figsize=(10,6))
sns.boxplot(x = 'R_to_sstation', y = 'sqm_price', data = data_after_2002, palette = 'Greens');

plt.xlabel('Residence distance to S-station', fontsize=10);
plt.ylabel('sqm price (DKK)', fontsize=10);

plt.xticks([0, 1, 2], ['r < 0.5 km', 'r > 0.5km and r < 1 km', 'r > 1 km']);
f.tight_layout()
f.savefig('sqm_distance_sstations.png');
                                                         
# creating boxplot for any stations                                                         
f, ax = plt.subplots(figsize=(10,6))
sns.boxplot(x = 'R_to_any_station', y = 'sqm_price', data = data_after_2002, palette = 'Greens');

plt.xlabel('Residence distance to any station', fontsize=10);
plt.ylabel('sqm price (DKK)', fontsize=10);

plt.xticks([0, 1, 2], ['r < 0.5 km', 'r > 0.5km and r < 1 km', 'r > 1 km']);
f.tight_layout()
f.savefig('sqm_distance_any_stations.png');


## Machine Learning

In [ ]:
dir = str(os.getcwd())
datafile = dir + "/boliga/data/analysis_data.csv"
df = pd.read_csv(datafile, index_col=0)
df = df[df['Date_of_sale'] > '2002']

### Making a list of municipalities

In [ ]:
dir = str(os.getcwd())
datafile = dir + "/boliga/data/analysis_data.csv"
df = pd.read_csv(datafile, index_col=0)

#seperating municipality
address = pd.DataFrame([a[0] + ' ' + a[-1][6:]\
           for a in df['Address'].str.split(',')])
header=['ad']
address.columns=header

municipalities = ['Frederiksberg C', 'Frederiksberg', 'København K', 'København V', 'København S', 'København N', 'København Ø']

address_list = []

for i in address['ad']: 
    for x in municipalities:
        if x in i:
            address_list.append(x)
            break

# Making a list of municipalities            
Municipality = pd.DataFrame(address_list)
Municipality.columns=['Municipality']

# Adding the list to the housing data
df['Municipality'] = Municipality
df
df = df.drop(['Sell_price', 'sqm_price','Date_of_sale','Address', 'location', 'latitude', 'longitude','m_distance_const', 'm_station_const', 's_distance_const',
       's_station_const', 's_station', 'm_station'], axis=1).dropna();
df = pd.get_dummies(df)
df = df.reset_index(drop=True)
y = df.z_sqm_price.to_numpy()
X = df.drop(['z_sqm_price', 'Type_ Villa ', 'Municipality_København K'], axis=1).to_numpy()
X = np.array(X, dtype=np.float64)
X, y;

### Splitting into test and train data

In [ ]:
# splitting into development (2/3) and test data (1/3)
X_dev, X_test, y_dev, y_test = train_test_split(X, y, test_size=1/3, random_state=34)

# splitting development into train (1/3) and validation (1/3)
X_train, X_val, y_train, y_val = train_test_split(X_dev, y_dev, test_size=1/2, random_state=52)

lambda_ = np.logspace(-1, 1, 22)
l1_ratio_ = np.logspace(0, 3, 22)
tol = 0.001

### OLS

In [ ]:
pipe_OLS = make_pipeline(PolynomialFeatures(degree = 3, include_bias=True), 
                           StandardScaler(),
                           LinearRegression())
                         
pipe_OLS.fit(X_dev, y_dev)
print(mse(pipe_OLS.predict(X_test),y_test), round(np.sqrt(mse(pipe_OLS.predict(X_test),y_test))))

### Lasso Regression

In [ ]:
pipe_lasso = make_pipeline(PolynomialFeatures(degree = 3, include_bias=True), 
                           StandardScaler(),
                           Lasso(max_iter = 10000, alpha=lambda_, tol=tol))

train_scores, test_scores = validation_curve(estimator=pipe_lasso,
                                             X=X_train, y=y_train,
                                             param_name='lasso__alpha',
                                             param_range=lambda_,
                                             scoring='neg_mean_squared_error',
                                             cv=5)

mse_score = pd.DataFrame({'Train':-train_scores.mean(axis=1),
                          'Validation':-test_scores.mean(axis=1),
                          'lambda':lambda_}).set_index('lambda')   

optimal_lambda_lasso = mse_score.Validation.nsmallest(1)
lambda_o_ = optimal_lambda_lasso.index

#testing
pipe_lasso = make_pipeline(PolynomialFeatures(degree = 3, include_bias=True), 
                           StandardScaler(),
                           Lasso(max_iter = 10000, alpha=optimal_lambda_lasso.index, tol=tol))

pipe_lasso.fit(X_dev, y_dev)
print(lambda_o_,' ',round(mse(pipe_lasso.predict(X_test),y_test),3), round(np.sqrt(mse(pipe_lasso.predict(X_test),y_test)),3))

### Ridge Regression

In [ ]:
pipe_ridge = make_pipeline(PolynomialFeatures(degree = 3, include_bias=True), 
                           StandardScaler(),
                           Ridge(alpha = l1_ratio_, tol=tol))

train_scores, test_scores = validation_curve(estimator=pipe_ridge,
                                             X=X_train, y=y_train,
                                             param_name='ridge__alpha',
                                             param_range=l1_ratio_,
                                             scoring='neg_mean_squared_error',
                                             cv=5)

mse_score = pd.DataFrame({'Train':-train_scores.mean(axis=1),
                          'Validation':-test_scores.mean(axis=1),
                          'lambda':l1_ratio_}).set_index('lambda')   

lambda_o_ = mse_score.Validation.nsmallest(1)
lambda_o_ = lambda_o_.index


#testing
pipe_ridge = make_pipeline(PolynomialFeatures(degree = 3, include_bias=True), 
                           StandardScaler(),
                           Ridge(alpha = lambda_o_, tol=tol))

pipe_ridge.fit(X_dev, y_dev)
print(lambda_o_,' ', round(mse(pipe_ridge.predict(X_test),y_test),3), round(np.sqrt(mse(pipe_ridge.predict(X_test),y_test)),3))

### LassoCV

In [ ]:
kfolds = KFold(n_splits=10)
folds = list(kfolds.split(X_dev, y_dev))

# outer loop: lambdas
mseCV = []
for lambda__ in lambda_:    
    # inner loop: folds
    mseCV_ = []    
    for train_idx, val_idx in folds :        
        # train model and compute MSE on test fold
        pipe_lassoCV = make_pipeline(PolynomialFeatures(degree=3, include_bias=False),
                                     StandardScaler(),
                                     Lasso(max_iter = 10000, alpha=lambda__, tol=tol))            
        X_train, y_train, = X_dev[train_idx], y_dev[train_idx]
        X_val, y_val = X_dev[val_idx], y_dev[val_idx] 
        pipe_lassoCV.fit(X_train, y_train)        
        mseCV_.append(mse(pipe_lassoCV.predict(X_val), y_val))
        
    # store result    
    mseCV.append(mseCV_) 
    
# convert to DataFrame
lambdaCV = pd.DataFrame(mseCV, index=lambda_)
lambdaCV['m'] = lambdaCV.mean(axis=1)
lambda_o_ = lambdaCV['m'].nsmallest(1).index

pipe_lassoCV = make_pipeline(PolynomialFeatures(degree=3, include_bias=False),
                                     StandardScaler(),
                                     Lasso(max_iter = 10000, alpha=lambda_o_, tol=tol))

pipe_lassoCV.fit(X_dev, y_dev)
print(lambda_o_,' ',round(mse(pipe_lassoCV.predict(X_test),y_test),3), round(np.sqrt(mse(pipe_lassoCV.predict(X_test),y_test)),3))

poly = PolynomialFeatures(3, False)
stdr = StandardScaler()
poly.fit(X_dev)
X_dev_ = poly.transform(X_dev)
X_test_ = poly.transform(X_test)
X_dev_ = stdr.fit_transform(X_dev_)
X_test_ = stdr.transform(X_test_)


lasso = Lasso(max_iter = 10000, alpha=lambda_o_, tol=tol)
lasso.fit(X_dev_ , y_dev)
round(mse(lasso.predict(X_test_), y_test),3) # is same as pipeline model above
print(lasso.score(X_test_, y_test))


X_columns = list(df.drop(['z_sqm_price', 'Type_ Villa ', 'Municipality_København K'], axis=1).columns)
X_columns = poly.get_feature_names(X_columns)

coefs = pd.DataFrame(lasso.coef_, index = X_columns)
coefs = coefs[coefs[0]!= 0]
coefs['abs'] = abs(coefs[0])
coefs['b'] = round(coefs[0],3)
coefs.to_csv('coefs.csv')


### RidgeCV

In [ ]:
# outer loop: lambdas
mseCV = []
for l1_ratio__ in l1_ratio_:    
    # inner loop: folds
    mseCV_ = []    
    for train_idx, val_idx in folds :        
        # train model and compute MSE on test fold
        pipe_ridgeCV = make_pipeline(PolynomialFeatures(degree = 3, include_bias=True), 
                           StandardScaler(),
                           Ridge(alpha = l1_ratio__, tol=tol))          
        X_train, y_train, = X_dev[train_idx], y_dev[train_idx]
        X_val, y_val = X_dev[val_idx], y_dev[val_idx] 
        pipe_ridgeCV.fit(X_train, y_train)        
        mseCV_.append(mse(pipe_ridgeCV.predict(X_val), y_val))    
        
    # store result    
    mseCV.append(mseCV_) 
    
# convert to DataFrame
lambdaCV = pd.DataFrame(mseCV, index=lambda_)
lambdaCV['m'] = lambdaCV.mean(axis=1)
lambda_o_ = lambdaCV['m'].nsmallest(1).index


pipe_ridgeCV = make_pipeline(PolynomialFeatures(degree = 3, include_bias=True), 
                           StandardScaler(),
                           Ridge(alpha = l1_ratio__, tol=tol))

pipe_ridgeCV.fit(X_dev, y_dev)
print(lambda_o_,' ', round(mse(pipe_ridgeCV.predict(X_test),y_test),3), round(np.sqrt(mse(pipe_ridgeCV.predict(X_test),y_test)),3))

### ElasticNet

In [ ]:
pipe_el = make_pipeline(PolynomialFeatures(include_bias=False), 
                        StandardScaler(),
                        ElasticNet(tol=.01))

gs = GridSearchCV(estimator=pipe_el, 
                  param_grid={'elasticnet__alpha':np.logspace(-1,1,22),
                              'elasticnet__l1_ratio':np.linspace(0,3,10)}, 
                  scoring='neg_mean_squared_error', 
                  n_jobs=8,
                  iid=False,
                  cv=10)

gs.fit(X_train, y_train)
gs.predict(X_train)


print(gs.best_params_)
print(lambda_o_,' ', round(mse(gs.predict(X_test),y_test),3), round(np.sqrt(mse(gs.predict(X_test),y_test)),3))


lambda_o_ = gs.best_params_
lambda_o_['elasticnet__alpha']

pipe_el = make_pipeline(PolynomialFeatures(include_bias=False), 
                        StandardScaler(),
                        ElasticNet(alpha = lambda_o_['elasticnet__alpha'],
                                   l1_ratio= lambda_o_['elasticnet__l1_ratio'],
                                   tol=.01))

pipe_el.fit(X_dev, y_dev)
pipe_el.predict(X_test)

round(mse(pipe_el.predict(X_test),y_test),3), round(np.sqrt(mse(pipe_el.predict(X_test),y_test)),3)